# [手把手教你完成一个数据科学小项目（7）：经纬度获取与BDP可视化](https://zhuanlan.zhihu.com/p/42959539)
请先阅读[“中国年轻人正带领国家走向危机”，这锅背是不背？](https://zhuanlan.zhihu.com/p/41880873)  一文，以对“手把手教你完成一个数据科学小项目”系列有个全局性的了解。

截至目前我们已经完成了[数据爬取](https://zhuanlan.zhihu.com/p/42060094)、[数据提取与IP查询](https://zhuanlan.zhihu.com/p/42151036)、[数据异常与清洗](https://zhuanlan.zhihu.com/p/42244913)、[评论数变化情况分析](https://zhuanlan.zhihu.com/p/42333272)、[省份提取与可视化](https://zhuanlan.zhihu.com/p/42406348)、[城市提取与可视化](https://zhuanlan.zhihu.com/p/42532934)，本文将调用百度地图 API 获取地理位置的经纬度，并使用 BDP 绘制动态热力图。
<img src='images/heat-map-BDP-1h-11FPS.gif'> 

## 读取数据
之前系列文章和代码的最后末尾均可自行保存每次操作后的数据，比如新创建的那些列等等。

In [1]:
import pandas as pd
df = pd.read_csv('Sina_Finance_Comments_All_20180811_Cleaned.csv',encoding='utf-8')
df.head(2)

,Unnamed: 0,Unnamed: 0.1,No,page,nick,time,content,area,ip,ip_loc,...,channel,hot,level,login_type,media_type,mid,stamp,time_ymd,time_mdh,cmntcount
0,0,20,21,2,我阿訇打钱,2018-08-11 17:15:13,[费解][费解][费解],青海海南藏族自治州,36.100.157.102,"青海省海南州 电信 * GeoIP: Hangzhou, Zhejiang, China *...",...,cj,0,5,0,0,5B6EA91F-24649D66-17A3D8EEA-8C5-8D1,1533978913,2018-08-11,08-11 17,3773
1,1,21,22,2,TKK_Questioning,2018-08-11 17:04:20,那你以为不单身就能避免？,安徽合肥,36.7.153.141,"安徽省合肥市 电信 * GeoIP: Hefei, Anhui, China * China...",...,cj,0,4,0,0,5B6EA690-2407998D-1343F7922-8C5-877,1533978260,2018-08-11,08-11 17,3772


## 百度地图
### 创建应用
本回使用百度地图开放平台的 API 获取经纬度数据，读者也可以试下用高德地图、谷歌地图等等实现。首先在[百度地图开放平台](http://lbsyun.baidu.com/apiconsole/key)（需登录操作）的“`控制台`”处点击“`创建应用`”；可以随意填写“`应用名称`”，比如：`地图经纬度`；在“`IP白名单`”处可按照提示填写 `0.0.0.0/0`，方便在不同的电脑上操作，然后点击“提交”即可创建成功。`应用列表`里生成了应用，而`访问应用（AK）`就是后面调用 API 时所需的参数。

### 获取经纬度
在`Web服务API`中找到所需的服务，本次是根据地理信息获取经纬度，于是`正/逆地理编码`，按照`服务文档`的说明，调用API。注意调用次数的限制。
[正/逆地理编码 -  百度地图Web服务API](http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding)    

In [5]:
import requests
def area2coor(area):
    ak = '换成你的ak' # 应用列表里访问应用（AK）的一串字符
    try:
        # http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding
        address = area
        url = 'http://api.map.baidu.com/geocoder/v2/?address=' + address  + '&output=json&ak=' + ak
        json_data = requests.get(url = url).json()    
        coor_loc = json_data['result']['location']
        return coor_loc
    except:
        return "nocoor" # 不换ak多半都是 'nocoor'
    
print(df.loc[0,'area'])    
area2coor(df.loc[0,'area'])

青海海南藏族自治州


{'lat': 36.284363803805, 'lng': 100.6240660940972}

## 经纬度
上面先测试下，能拿到经纬度后，创建经纬度列

In [257]:
%%time
df['coor_loc'] = df.area.apply(area2coor)
df.coor_loc

0       {'lng': 100.6240660940972, 'lat': 36.284363803...
1       {'lng': 117.28269909168304, 'lat': 31.86694226...
2       {'lng': 112.5508635890553, 'lat': 37.890277053...
3       {'lng': 113.30764967515182, 'lat': 23.12004910...
4       {'lng': 114.0259736573215, 'lat': 22.546053546...
5       {'lng': 111.720663546476, 'lat': 27.6958640523...
6       {'lng': 102.89915972360397, 'lat': 30.36748093...
7       {'lng': 119.95720242066378, 'lat': 29.15949412...
8       {'lng': 108.29723355586638, 'lat': 22.80649293...
9       {'lng': 120.2193754157201, 'lat': 30.259244461...
10      {'lng': 113.12029208572514, 'lat': 36.20166438...
11      {'lng': 121.48789948569473, 'lat': 31.24916171...
12      {'lng': 114.31620010268132, 'lat': 30.58108412...
13      {'lng': 117.18810662317686, 'lat': 34.27155343...
14      {'lng': 121.48789948569473, 'lat': 31.24916171...
15      {'lng': 119.91960601619071, 'lat': 32.47605327...
16      {'lng': 121.48789948569473, 'lat': 31.24916171...
17      {'lng'

确定`'coor_loc'`列是字典形式后，就可以直接从字典中拿到经度和维度了。

In [270]:
coor1 = df.coor_loc.values.tolist()
type(coor1),coor1

(list,
 [{'lat': 36.284363803805, 'lng': 100.6240660940972},
  {'lat': 31.86694226068694, 'lng': 117.28269909168304},
  {'lat': 37.89027705396754, 'lng': 112.5508635890553},
  {'lat': 23.12004910207623, 'lng': 113.30764967515182},
  {'lat': 22.546053546205247, 'lng': 114.0259736573215},
  {'lat': 27.695864052356377, 'lng': 111.720663546476},
  {'lat': 30.36748093795755, 'lng': 102.89915972360397},
  {'lat': 29.159494120760925, 'lng': 119.95720242066378},
  {'lat': 22.80649293560261, 'lng': 108.29723355586638},
  {'lat': 30.259244461536102, 'lng': 120.2193754157201},
  {'lat': 36.20166438574343, 'lng': 113.12029208572514},
  {'lat': 31.24916171001514, 'lng': 121.48789948569473},
  {'lat': 30.58108412692075, 'lng': 114.31620010268132},
  {'lat': 34.27155343109188, 'lng': 117.18810662317686},
  {'lat': 31.24916171001514, 'lng': 121.48789948569473},
  {'lat': 32.47605327483028, 'lng': 119.91960601619071},
  {'lat': 31.24916171001514, 'lng': 121.48789948569473},
  {'lat': 32.05723550180587,

## 拆分经度纬度
选出非'nocoor'的数据，再分别拿到经度和纬度，然后就可以导出数据，以便后面在BDP里操作。

In [271]:
df_coor = df[df['coor_loc'] != 'nocoor']
df_coor['lng'] = df_coor['coor_loc'].apply(lambda x: x['lng']) # 经度
df_coor['lat'] = df_coor['coor_loc'].apply(lambda x: x['lat']) # 纬度
df_coor[['lng','lat']]

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,lng,lat
0,100.624066,36.284364
1,117.282699,31.866942
2,112.550864,37.890277
3,113.307650,23.120049
4,114.025974,22.546054
5,111.720664,27.695864
6,102.899160,30.367481
7,119.957202,29.159494
8,108.297234,22.806493
9,120.219375,30.259244


## 保存数据

In [272]:
df_coor.to_csv('Sina_Finance_Comments_All_20180811_toBDP.csv', encoding='utf-8', line_terminator='\r\n')

## 动态热力图
古柳以前也用过 BDP，所以这回拿到数据后，就想着间隔近一年的时间后重新绘制动态热力图，虽则早已生疏了，但以前机智的写过一篇“使用手册”：[（送福利）BDP绘制微博转发动态热力图](https://zhuanlan.zhihu.com/p/29557747)，于是按照文中步骤很快就重新捡回并制作出来了。

具体步骤就不截图演示了，更详细的步骤请参考上面给出的文章，内含爬取的微博转发数据集，可供把玩（用Gephi一则热门微博的14层转发网络图谱：[《Gephi绘制微博转发图谱：以“@老婆孩子在天堂”为例》](https://zhuanlan.zhihu.com/p/29557827)）
<img src='https://pic1.zhimg.com/80/v2-46ee907f5ef96db00aa12456728620d8_hd.jpg'>
<img src='https://pic1.zhimg.com/v2-244681d95cdd619de6d7939a339ba6b5_r.jpg'>

此处仅记录大致操作步骤如下：
- 网上搜索：[BDP个人版](https://me.bdp.cn/home.html)，注册账号以便使用；
- 点击“数据源”，点击“立即添加”，点击“CSV上传”，按照跳出的页面，上传本地对应的CSV文件，“逗号”分割，确定后，等待上传成功后，就能看到数据，此处将相应的时间列，设定为日期，否则后面动态展示时可能会出错。点击下一步，改不改文件名，目录，随意，之后下一步，完成数据上传；
- 点击菜单栏右上角“新建图表”，选择“经纬度地图”后确定；
- 经度选择上传的CSV数据里的“lng”列，纬度选择“lat”列，坐标系选择为百度地图；
- 将工作表中文件拖曳到图层里，就能在地图上加载出数据，非常简单地拿到了地图；

更改设置参数，以便录制 GIF 时展示效果更佳：
- 热力半径：8像素
- 时间粒度：按时
- 时间间隔：2小时 / 1小时
- 自定义速度：FPS：8 / 12 

可根据数据量、数据展示的效果、以及自身的要求自行修改。最后就拿到了文章评论的动态热力图，还是蛮酷的。

<img src='images/heat-map-BDP-1h-11FPS.gif'> 